In [ ]:
import sys
sys.path.append('/home/luyao/PR_get/INTRUDE')

from git import *

with open('../data/clf/second_nondup.txt') as f:
    pairs = f.readlines()

w = 0

out = open('../data/multi_commits_second_nondup.txt', 'w')

pairs = sorted(pairs, key=lambda x: x.split()[0])
last_repo = None

for pair in pairs:
    repo, num1, num2 = pair.split()
    
    #print(repo, num1, num2)
    if repo != last_repo:
        last_repo = repo
        init_model_with_repo(repo)
    
    p1 = get_pull(repo, num1)
    p2 = get_pull(repo, num2)
    
    cl1 = get_pull_commit(p1)
    cl2 = get_pull_commit(p2)
    
    
    #t = get_pr_sim_vector(p1, p2)
    #o = m.predict_proba([t])[0][1]
    
    if (len(cl1) == 1) and (len(cl2) == 1):
        continue
        
    if (len(cl1) == 0) or (len(cl2) == 0):
        continue
    
    if (len(cl1) > 100) or (len(cl2) > 100):
        continue

    if check_large(p1) or check_large(p2):
        continue
    
    # if o >= 0.8:
    print(pair.strip(), o, file=out)
    
    w += 1
    if w % 10 == 0:
        print(w)

out.close()
print(w)


In [18]:

treshold = 0.9

# file = '../evaluation/multi_commits_rly_false_pairs_result.txt'
file = '../evaluation/multi_commits_msr_result.txt'

with open(file) as f:
    ps = f.readlines()


save = []
for p in ps:
    p1, hist = p.strip()[:-1].split('[')
    r, n1, n2, ori, det, delta, zz = p1.split()
    
    if det == '2':
        continue

    hist = [float(h) for h in hist.split(',')]
    
    
    if len(hist) == 1:
        continue
    
    
    s = 0
    for i in range(len(hist)):
        if hist[i] >= treshold:
            s = len(hist) - i - 1
            break
    save.append(s)

print(len(save))
no_zero = 0
for i in save:
    if i > 0:
        no_zero += 1
print('no_zero=', no_zero / len(save))
print(sum(save))
print(sum(save) / len(save))      

211
no_zero= 0.5165876777251185
316
1.4976303317535544


In [ ]:
from main import *
from git import *
from comparer import *

m = classify()


# t = get_sim_vector(get_pull('JuliaLang/julia', '12014'), get_pull('JuliaLang/julia', '11721'))
t = get_sim_vector(get_pull('JuliaLang/julia', '16942'), get_pull('JuliaLang/julia', '16917'))

print(m.predict_proba([t])[0])

In [ ]:
# run detection

from comparer import *
from git import *
from main import *
from util import localfile_tool

repo = 'angular/angular.js'
num1 = '6341'
num2 = '6167'

m = classify()

with open('data/msr_positive_pairs.txt') as f:
    pairs = f.readlines()

label = 1

num_ok, num_total = 0, 0

X, y = [], []

for pair in pairs:
    repo, num1, num2 = pair.split()
    
    p1 = get_pull(repo, num1)
    p2 = get_pull(repo, num2)
    
    '''
    if (p1["merged_at"] is not None) and (p2["merged_at"] is not None):
        continue
    '''
    
    cl1 = get_pull_commit(p1)
    cl2 = get_pull_commit(p2)
    
    '''
    if (len(cl1) == 1) and (len(cl2) == 1):
        continue
    '''
    
    if (len(cl1) >= 100) or (len(cl2) >= 100):
        continue
    if check_too_big(p1) or check_too_big(p2):
        continue
    
    max_s, max_p1, max_p2, max_vet = -1, None, None, None
    
    for c1 in cl1:
        for c2 in cl2:
            if c1['url'] != c2['url']:
                vet = get_sim_vector_on_commit(c1, c2)
                t = m.predict_proba([vet])[0][1]
                if t > max_s:
                    max_s, max_p1, max_p2, max_vet = t, c1['html_url'], c2['html_url'], vet
    
    X.append(vet)
    y.append(label)
    
    num_total += 1
    if num_total % 100 == 0:
        print(num_total)
    
    
    '''
    if m.predict([get_sim_vector(p1, p2)])[0] == 1:
        ok = True
    '''
    # print(m.predict_proba([get_sim_vector(p1, p2)])[0][1])
    '''
    if max_s >= 0.8:
        print(max_s, max_p1, max_p2, p1['html_url'] + '/commits', p2['html_url'] + '/commits', sep='\n')
        print(len(cl1), len(cl2))
    '''
    
    '''
    if ok:
        num_ok += 1
    num_total += 1
    if num_total >= 500:
        break
    print(num_total)
    '''

    
# print(1.0 * num_ok / num_total)
    

out = 'data/msr_positive_pairs_feature_vector_commit_g'

X_path = out + '_X.json'
y_path = out + '_y.json'

localfile_tool.write_to_file(X_path, X)
localfile_tool.write_to_file(y_path, y)
    